In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features, reduce_mem_usage
import datetime

from tqdm import tqdm
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

feat_no = '110_ker'

### Data Load

In [3]:
# df_hist = pd.read_csv('../input/historical_transactions.csv')
# df_new = pd.read_csv('../input/new_merchant_transactions.csv')
# df_hist = reduce_mem_usage(df_hist)
# df_new = reduce_mem_usage(df_new)
# utils.to_df_pkl(df=df_hist, path='../input', fname='historical_transactions')
# utils.to_df_pkl(df=df_new, path='../input', fname='new_merchant_transactions')

In [3]:
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
df_train.set_index(key, inplace=True)
df_test.set_index(key, inplace=True)
df_hist = utils.read_df_pkl('../input/histori*0*')
df_new = utils.read_df_pkl('../input/new_mer*0*')

df_train = reduce_mem_usage(df_train)
df_test  = reduce_mem_usage(df_test )
df_hist  = reduce_mem_usage(df_hist )
df_new   = reduce_mem_usage(df_new  )

historical_transactions = df_hist
new_transactions = df_new
train = df_train
test = df_test


100%|██████████| 3/3 [00:07<00:00,  2.47s/it]

100%|██████████| 3/3 [00:00<00:00,  4.96it/s]


Mem. usage decreased to  4.04 Mb (56.2% reduction)
Mem. usage decreased to  2.24 Mb (52.5% reduction)
Mem. usage decreased to 1971.22 Mb (0.0% reduction)
Mem. usage decreased to 129.17 Mb (0.0% reduction)


In [4]:
for df in [historical_transactions, new_transactions]:
    df['category_1'] = df['category_1'].map({'Y':1, 'N':0}) 
#     df['category_2'].fillna(1.0,inplace=True)
#     df['category_3'].fillna('A',inplace=True)
#     df['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace=True)
    df['authorized_flag'] = df['authorized_flag'].map({'Y':1, 'N':0})

In [5]:
historical_transactions = pd.get_dummies(historical_transactions, columns=['category_2', 'category_3'])
new_transactions = pd.get_dummies(new_transactions, columns=['category_2', 'category_3'])

historical_transactions = reduce_mem_usage(historical_transactions)
new_transactions = reduce_mem_usage(new_transactions)

agg_fun = {'authorized_flag': ['mean']}
auth_mean = historical_transactions.groupby(['card_id']).agg(agg_fun)
auth_mean.columns = ['_'.join(col).strip() for col in auth_mean.columns.values]
auth_mean.reset_index(inplace=True)

authorized_transactions = historical_transactions[historical_transactions['authorized_flag'] == 1]
historical_transactions = historical_transactions[historical_transactions['authorized_flag'] == 0]

Mem. usage decreased to 1527.00 Mb (20.3% reduction)
Mem. usage decreased to 99.22 Mb (20.9% reduction)


In [13]:
def aggregate_transactions(history):
    
    history.loc[:, 'purchase_date'] = pd.DatetimeIndex(history['purchase_date']).\
                                      astype(np.int64) * 1e-9
    
#     agg_func = {
#     'category_1': ['sum', 'mean'],
#     'category_2_1.0': ['mean'],
#     'category_2_2.0': ['mean'],
#     'category_2_3.0': ['mean'],
#     'category_2_4.0': ['mean'],
#     'category_2_5.0': ['mean'],
#     'category_3_A': ['mean'],
#     'category_3_B': ['mean'],
#     'category_3_C': ['mean'],
#     }
    agg_func = {'authorized_flag': ['mean']}
    
    agg_history = history.groupby(['card_id']).agg(agg_func)
    agg_history.columns = ['_'.join(col).strip() for col in agg_history.columns.values]
    agg_history.reset_index(inplace=True)
    
    return agg_history

In [14]:
history = aggregate_transactions(historical_transactions)
history.columns = ['hist_' + c if c != 'card_id' else c for c in history.columns]
history[:5]

,card_id,hist_authorized_flag_mean
0,C_ID_00007093c1,0
1,C_ID_0001238066,0
2,C_ID_0001506ef0,0
3,C_ID_0001793786,0
4,C_ID_000183fdda,0


In [15]:
authorized = aggregate_transactions(authorized_transactions)
authorized.columns = ['auth_' + c if c != 'card_id' else c for c in authorized.columns]
authorized[:5]

,card_id,auth_authorized_flag_mean
0,C_ID_00007093c1,1
1,C_ID_0001238066,1
2,C_ID_0001506ef0,1
3,C_ID_0001793786,1
4,C_ID_000183fdda,1


In [9]:
new = aggregate_transactions(new_transactions)
new.columns = ['new_' + c if c != 'card_id' else c for c in new.columns]
new[:5]

,card_id,new_category_1_sum,new_category_1_mean,new_category_2_1.0_mean,new_category_2_2.0_mean,new_category_2_3.0_mean,new_category_2_4.0_mean,new_category_2_5.0_mean,new_category_3_A_mean,new_category_3_B_mean,new_category_3_C_mean
0,C_ID_00007093c1,0,0.000000,0.500000,0.000000,0.50000,0.0,0.000000,0.0,1.000000,0.000000
1,C_ID_0001238066,2,0.076923,0.769231,0.000000,0.00000,0.0,0.115385,0.0,0.807692,0.153846
2,C_ID_0001506ef0,0,0.000000,0.000000,0.000000,1.00000,0.0,0.000000,1.0,0.000000,0.000000
3,C_ID_0001793786,0,0.000000,0.483871,0.258065,0.16129,0.0,0.032258,1.0,0.000000,0.000000
4,C_ID_000183fdda,0,0.000000,0.000000,0.000000,1.00000,0.0,0.000000,0.0,0.545455,0.363636


In [10]:
train = pd.merge(train, history, on='card_id', how='left')
test = pd.merge(test, history, on='card_id', how='left')

train = pd.merge(train, authorized, on='card_id', how='left')
test = pd.merge(test, authorized, on='card_id', how='left')

train = pd.merge(train, new, on='card_id', how='left')
test = pd.merge(test, new, on='card_id', how='left')

In [18]:
prefix = '110_ker'
prefix = '310_ker'
# prefix = '410_ker'
df_feat = pd.concat([train, test], axis=0)
df_feat.columns = [col.replace('.', '_') for col in df_feat.columns]

for col in df_feat.columns:
    if not(col.count('category')):continue
        
    try:
#         feature = df_feat[col].fillna(-1).astype('float32').values
        feature = df_feat[col].astype('float32').values
    except TypeError:
        print(col)
        sys.exit()

    utils.to_pkl_gzip(path = f'../features/1_first_valid/{prefix}_{col}@', obj=feature)